## Apache Logs

This is a study of analyzing apache logs, looking for normal and abnormal usage.

Prerequisites
 - python 3.x
 - python packages
   - pip3 install pandas
   - pip3 install scikit-learn

Sample data
- http://indeedeng.github.io/imhotep/files/nasa_19950801.tsv (saved in 'rawData/nasa_19950801.tsv')
- http://indeedeng.github.io/imhotep/files/nasa_19950630.22-19950728.12.tsv.gz
- http://indeedeng.github.io/imhotep/files/nasa_19950731.22-19950831.22.tsv.gz


In [8]:
import pandas as pd
apache_log_path = 'rawData/nasa_19950801.tsv'
apache_log_data = pd.read_csv(apache_log_path, sep='\t', header=0)
apache_log_data.columns

Index(['host', 'logname', 'time', 'method', 'url', 'response', 'bytes',
       'referer', 'useragent'],
      dtype='object')